# The Case for Learned Index Structures

## Introduction
* Key insight: Indexes are models.
* The current (non ml) ones are general and not exploiting the patterns in the actual data being indexed.
    * Real world data follows some (complex) distribution. We can learn this distribution to make better predictions.
* Introduces the idea of using ML in systems where it has not been traditionally applied.
    * Jeff Dean talk at NIPS.

## Types of Indexes
The paper discusses a few different types of indexes which they try to improve by using neural nets to learn the indexes.
* Range Indexes
* Point Indexes
* Existence Indexes

### Main difficulties
* How to provide the same guarantees as the non ml models give?

## Range Index
* Used when we want a mapping from a lookup key to a position in a sorted array of data records with the guarantee that the key of the record at the position is equal or higher than the lookup key.
* E.g. B-tree index
    * Map key to a page of records
    * B-trees can be seen as regression trees, predict position with min- and max-error.
    * Error is in how many positions off the prediction was.
    * For B-trees, position predictions have min error of 0 and max error of page size.

<img src="figs/learned_indexes/btree.png" width="30%">

### Learned Range Index
* Want learned model $f$ to given key, predict position with some min- and max-error.
* When training the learned model, keep track of the min- ($err_{min}$) and max-error ($err_{max}$) of it.
* At prediction $f(key)$, search positions $[f(key) - err_{min}, f(key) + err_{max}]$ which has the same guarantees as a B-tree.
* This model can be any type of regression model, like linear regression or a neural net.
* An important insight is that range index models approximates the CDF of keys.
    * $cdf(key) = p(x \leq key)$, $cdf(key) \in [0, 1]$
    * Position prediction is then $p = cdf(key) * N$ where $N$ is number of keys.

<img src="figs/learned_indexes/range_learned_idx.png" width="30%">

#### First approach: Naive Model
* Small MLP trained to predict position given timestamps in web server log data.
* Good approximation of general shape of CDF but problems at the individual sample level. The "last mile" accuracy is an issue.
* Optimizing for average error isn't optimal in this case where we need small min- and max-error to find a record fast.

#### Second approach: Recursive Model
* Using just one model, it's easy to reduce large errors in prediction but hard for the very small ones. "Last mile" accuracy.
* Instead use a hierarchy of models (ordered by stages) with different responsibilities. Mixture of experts.
* Each sub model takes key as input and predicts position which is used to pick the next sub model in the next stage $l$ by $\lfloor M_l f_{l-1}(x)/N \rfloor$.
    * Basically divide the keyspace in $M_l$ parts and pick the next sub model responsible for the space the previous prediction fell in.
    * The picked model then also takes the key as input and so on.
* Last stage makes actual prediction.
* Each sub model is then responsible for different subspaces of the key space at increasing resolution.
    * The ones in the bottom is good at predicting within a small subspace of keys. Solves the "last mile" problem.
* Training is done by minimizing the squared error for the used sub models for a given prediction.
    * Trained stage by stage.
    * No gradient propagation between stages. Each sub model independently trained on (key, position) pairs in the relevant space.
* Each sub model can be different and even be a B-tree.
    * E.g. First some neural networks and then B-trees in the bottom stage.

<img src="figs/learned_indexes/rmi.png" width="50%">

* Results for Map data set (keys are longitude coordinates).
    * They also tried different search strategies within each position interval returned by the learned models.
    * The models tried here are two stage with an initial mlp with 0-2 hidden layers and then a lot of linear regressors.
    

<img src="figs/learned_indexes/map_data_rmi.png" width="70%">

#### Note on string keys
* They try using fixed length strings (padded with 0 or truncated) by just converting to ascii values of individual characters.
* This key is then a vector.
* Want to model interactions between characters. Important for modeling CDF.
* Open for future research. Could use RNNs and/or CNNs.

#### Note on insertions
* For B-trees, can set min- and max-fillfactor of pages to leave some space for insertions.
* Can possibly do the same but smarter by considering the CDF to know where to leave more space.
* Also a trade off with the "last mile" accuracy. This is basically overfitting so with very high accuracy it would be harder to leave space for new keys.

## Point Index
* Used when we want to look up a record in an unordered array.
* E.g. a hash map.
    * Hash function to map key to random positions in this array.
    * Collisions are a problem that maybe can be alleviated by learning a model that more uniquely maps keys to positions.
* Hard to beat hash map speedwise (with enough space allotted) but can save space.
* Knowing something about the CDF is helpful in this case.
    * Can define hash function $h(k) = f(k) * M$ where $M$ is size of hash map.
    * Can use the same recursive model architecture from before to learn the CDF.

<img src="figs/learned_indexes/point_index_results.png" width="50%">

## Existence Index
* Used when we just want to know if a key is in a set.
* E.g. bloom filter.
    * M sized bit array.
    * k hash functions.
    * Insertion of $e$, compute $hash_k(e) \forall_k$ and set those indexes to 1.
    * To test for inclusion of $e$, compute $hash_k(e) \forall_k$. If any bit is zero, it's not in the set.
    * Guaranteed no false negatives. Might have false positives.
    * Can lower false positive rate by sacrificing space
* Learned Existence Index as binary classifier.
    * Train a neural net for this binary classification task
    * Tune the false positive rate by setting a probability threshold $\gamma$ for predicting that a key is in the set.
    * For all the false negatives from this model, use a normal bloom filter. Then we get the same guarantees but likely using less space.

## Discussion
* When doesn't it work well?
* Consequences of using this type of models. Harder to debug?
* Focus here was on read only? They sketched some ideas for how to do this for write heavy applications as well.
    * They say (re)indexing of B-tree is analogous to training of the RMI model. Similar time for a fit?
* Key takeaways?
    * "we believe that the idea of replacing core components of a data management system through learned models..."